In [1]:
from pandas import Series, DataFrame
import pandas as pd
import numpy as np
import nltk
import re
from nltk.stem import WordNetLemmatizer
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
import sklearn.metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import grid_search
from sklearn.linear_model import LogisticRegression

In [2]:
traindf = pd.read_json("train.json")
testdf = pd.read_json("test.json")
wnl = WordNetLemmatizer()
def lemmatize_each_row(x):
    y = []
    for each in x:
        y.append(wnl.lemmatize(each.lower()))
    return y

In [3]:
traindf['lemmatized_ingredients_list'] = traindf.apply(lambda row: lemmatize_each_row(row['ingredients']), axis=1)
all_ingredients_lemmatized = []
for ingredients_lists in traindf.ingredients:
    for ingredient in ingredients_lists:
        all_ingredients_lemmatized.append(wnl.lemmatize(ingredient.lower()))
all_ingredients_lemmatized = set(all_ingredients_lemmatized)
testdf['lemmatized_test_ingredients_list'] = testdf.apply(lambda row: lemmatize_each_row(row['ingredients']), axis=1)
all_ingredients_lemmatized_test = []
for ingredients_lists in testdf.ingredients:
    for ingredient in ingredients_lists:
        all_ingredients_lemmatized_test.append(wnl.lemmatize(ingredient.lower()))
all_ingredients_lemmatized_test = set(all_ingredients_lemmatized_test)

In [4]:
all_ingredients_union = all_ingredients_lemmatized | all_ingredients_lemmatized_test

In [5]:
vect = TfidfVectorizer(preprocessor=lambda x: x, tokenizer=lambda x: x, sublinear_tf=True, max_df=0.5, analyzer='word', stop_words='english', vocabulary=all_ingredients_union)
tfidf_matrix = vect.fit_transform(traindf['lemmatized_ingredients_list'])
predictor_matrix = tfidf_matrix
target_classes = traindf['cuisine']

In [6]:
vect_test = TfidfVectorizer(preprocessor=lambda x: x, tokenizer=lambda x: x, sublinear_tf=True, max_df=0.5, analyzer='word', stop_words='english', vocabulary=all_ingredients_union)
tfidf_matrix_test = vect_test.fit_transform(testdf['lemmatized_test_ingredients_list'])
predictor_matrix_test = tfidf_matrix_test

In [7]:
from sklearn.naive_bayes import BernoulliNB

In [8]:
clf = BernoulliNB()
clf.fit(predictor_matrix, target_classes)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [9]:
predicted_classes = clf.predict(predictor_matrix_test)

In [10]:
testdf['cuisine'] = predicted_classes
submission=testdf[['id' ,  'cuisine' ]]
submission.to_csv("Submission.csv",index=False)